# OpenAI Agent with LlamaIndex

## Install Dependencies

In [1]:
!pip install uv
!uv pip install -qU xpander-sdk llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

## Setup API Keys


In [2]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

## Import libraries and setup LlamaIndex

In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4o")

# Set up Phoenix

In [4]:
import phoenix as px
session = px.launch_app()

/Users/dudu/Developer/xpander-lab/projects/xpander-agents-hub/agents/llama-index/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [5]:
from xpander_sdk import XpanderClient, LLMProvider, OpenAISupportedModels
# load the client
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, organization_id="")
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)
# create execution
xpander_agent.invoke_agent("Get only the longest readable tag")

# init the memory
xpander_agent.memory.select_llm_provider(llm_provider=LLMProvider.OPEN_AI)
xpander_agent.memory.initialize_thread(input=xpander_agent.execution.input_message,instructions=xpander_agent.instructions)
print(xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI))
print("Input:",xpander_agent.execution.input_message.content)
print("General:",xpander_agent.instructions._delegates[0].general)
print("Goal:",xpander_agent.instructions._delegates[0].goal)
print("Role:",xpander_agent.instructions._delegates[0].role)
xpander_agent.memory.initialize_thread(input=xpander_agent.execution.input_message,instructions=xpander_agent.instructions)
print("-"*100)
print("Messages object:")
for message in xpander_agent.memory.messages:
    print("-"*100)
    print("Role:",message.role)
    print("Tool Call ID:",message.tool_call_id)
    print("Tool Calls:",message.tool_calls)
    print("Content:",message.content)
print("-"*100)

I0000 00:00:1738109811.500656 7318381 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
b'\x1b\x1b!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software has not been tested with node v23.4.0.                                                                !!\x1b\x1b\n'
b'\x1b\x1b!!  Should you encounter odd runtime issues, please try using one of the supported release before filing a bug report.  !!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software is currently running on node v23.4.0.                                                                 !!\x1b\x1b\n'
b'\x1b\x

[{'type': 'function', 'function': {'name': 'LinkedInProfileServiceConvertProfileUrlToEmail', 'description': "Attempts to retrieve email addresses associated with a given LinkedIn profile URL. This operation is valuable for obtaining contact information. If the profile URL isn't provided, run SearchProfilesByCriteria first to get the profile URL. Use this operation for lead generation or to establish direct contact with LinkedIn users when you have their profile URL but need their email address.", 'parameters': {'type': 'object', 'properties': {'bodyParams': {'type': 'object', 'properties': {}, 'required': []}, 'queryParams': {'type': 'object', 'properties': {'url': {'type': 'string', 'description': "LinkedIn profile URL (e.g., 'https://www.linkedin.com/in/taylorotwell')."}}, 'required': ['url']}, 'pathParams': {'type': 'object', 'properties': {}, 'required': []}}, 'required': ['bodyParams', 'queryParams', 'pathParams']}}}, {'type': 'function', 'function': {'name': 'LinkedInProfileServi

In [6]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Load Documents

In [7]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [8]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, show_progress=True)
    uber_index = VectorStoreIndex.from_documents(uber_docs, swow_progress=True)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

Now create the query engines.

In [9]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3, llm=llm)
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [10]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [11]:
llama_agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [12]:
response = llama_agent.chat("Who had more profit in 2021, Lyft or Uber?")
print(str(response))

> Running step 5a5d811d-6b4c-417e-be02-5f88fb7c6f3d. Step input: Who had more profit in 2021, Lyft or Uber?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's profit in 2021?"}
Observation: Lyft did not report a profit in 2021. Instead, the company had a net loss of $1.0 billion for the year.
> Running step e65607ea-23cd-4831-ae4b-1bca49ebbdab. Step input: None
Thought: I have information about Lyft's financial performance. Now, I need to find out Uber's profit for 2021 to compare.
Action: uber_10k
Action Input: {'input': "What was Uber's profit in 2021?"}
Observation: Uber did not report a profit in 2021. Instead, it recorded a net loss attributable to Uber Technologies, Inc. of $496 million.
> Running step 23b2af42-16c6-4be1-acef-4f07b24d392a. Step input: None
Thought: I have gathered the necessary information about both companies' financial performances in 2021. Lyft ha

In [13]:
from openai import OpenAI
from xpander_sdk import XpanderClient, LLMProvider, OpenAISupportedModels
openai_client = OpenAI(api_key=OPENAI_API_KEY)
# create execution
xpander_agent.invoke_agent("Who had more profit in 2021, Lyft or Uber?")

# init the memory
xpander_agent.memory.select_llm_provider(llm_provider=LLMProvider.OPEN_AI)
xpander_agent.memory.initialize_thread(input=xpander_agent.execution.input_message,instructions=xpander_agent.instructions)

while not xpander_agent.is_finished():
    print("-"*100)
    print("Messages:",xpander_agent.memory.retrieve_messages())
    print("Tools:",xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI))
    print("-"*100)
    response = openai_client.chat.completions.create(
                model= OpenAISupportedModels.GPT_4_O,
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI),
                tool_choice="auto",
                temperature=0.0
        )
            
    # add messages directly from the LLM response
    xpander_agent.memory.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump(),llm_provider=LLMProvider.OPEN_AI)
    # run tools
    xpander_agent.run_tools(tool_calls=tool_calls)

# result (re fetch execution result)
# IMPORTANT: LAST TOOL IS xpfinish-agent-execution-finished WHICH IS AGENT-END with PARSING and may be slower due to inference times
execution_result = xpander_agent.execution.fetch(agent=xpander_agent,execution_id=xpander_agent.execution.id)
print("status", execution_result.status)
print("result", execution_result.result)

----------------------------------------------------------------------------------------------------
Messages: [{'role': 'system', 'content': 'Your General instructions: Professional\n        IMPORTANT: when making a sub-task (with input_task), make sure to append all related \n        information relevant for the sub-task. For instance, if the sub-task is about sending \n        an email with tags, ensure the tags and any required data are included in the input_task.\n        \n        - Execute tools as needed to complete tasks.  \n        - If a tool fails **3 times**, execution stalls, or no tool is called for an extended period, stop immediately and report the issue along with any relevant results or errors.\n        \nYour Role instructions: You are responsible for summarizing Linkedin Profiles\nYour Goal instructions: Return concise profile summary\nIMPORTANT: When done or unable to proceed after attempts, use "xpfinish-agent-execution-finished" to mark success or failure.'}, {'

b'{"stdout":"YWRkaW5nIG1lc3NhZ2VzIHRvIHRocmVhZCAwNjhjMzFhOS02MjNiLTRhNmMtOTc2Ni1hM2ViNzM4NzEyYWYgWwogIHsKICAgIHJvbGU6IBtbMzJtJ2Fzc2lzdGFudCcbWzM5bSwKICAgIGNvbnRlbnQ6IBtbMzJtJ0NvdWxkIHlvdSBwbGVhc2UgcHJvdmlkZSBtb3JlIGNvbnRleHQgb3Igc3BlY2lmeSB0aGUgc291cmNlIG9yIHR5cGUgb2YgdGFncyB5b3UgYXJlIHJlZmVycmluZyB0bz8gVGhpcyB3aWxsIGhlbHAgbWUgYXNzaXN0IHlvdSBtb3JlIGVmZmVjdGl2ZWx5LicbWzM5bSwKICAgIHRvb2xDYWxsczogG1s5MG11bmRlZmluZWQbWzM5bQogIH0KXQo="}\n'
b'{"stdout":"YWRkaW5nIG1lc3NhZ2VzIHRvIHRocmVhZCAwNjhjMzFhOS02MjNiLTRhNmMtOTc2Ni1hM2ViNzM4NzEyYWYgWyB7IHJvbGU6IBtbMzJtJ2Fzc2lzdGFudCcbWzM5bSwgY29udGVudDogG1s5MG11bmRlZmluZWQbWzM5bSwgdG9vbENhbGxzOiBbIBtbMzZtW09iamVjdF0bWzM5bSBdIH0gXQo="}\n'
b'{"stdout":"cnVubmluZyB0b29sIHhwZmluaXNoLWFnZW50LWV4ZWN1dGlvbi1maW5pc2hlZCBvbiBhZ2VudCA4NjBmYzcyNS05NzI4LTQ2ZjctOGIwYi1jNDU3NzRmNDRmMTkgd2l0aCBleGVjdXRpb24gZWJkNmE0YWUtNjcxNC00YzZiLTk0YWItOWI1MDBmNzE2NjYwCg=="}\n'


In [15]:
from xpander_sdk import ExecutionStatus
while execution_result.status == ExecutionStatus.PENDING:
    execution_result = xpander_agent.execution.fetch(agent=xpander_agent, execution_id=xpander_agent.execution.id)
print("status", execution_result.status)
print("result", execution_result.result)

KeyboardInterrupt: 